In [119]:
# Imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree

# Read df
df = pd.read_excel('input.xlsx')
df = df[['producto','precio']]

# Input values
productos = df['producto'].values.tolist()
precio_compra = df['precio'].values.tolist()
productos_buscable = ['-'.join(p.strip().split(' ')) for p in productos]


# Obtencion del precio en Mercado libre
n_mean = 4

base_url = 'https://listado.mercadolibre.com.ar/'

lista_precios = []
no_buscables = []

for i, prod in enumerate(productos_buscable[:10]):
  r = requests.get(base_url+prod)
   
  if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Nombres
    nombres = soup.find_all('h2', {'class':'ui-search-item__title shops__item-title'})
    nombres = [n.text for n in nombres]
    
    # Precios
    precios = soup.find_all('span', {'class':'price-tag-fraction'})
    precios = [int(i.text.replace('.','')) for i in precios[:n_mean]]

    # Distancia de Jaccard
    precios_correctos = [] 
    for i2, n in enumerate(nombres):
        if string_similarity(productos[i], n) > 0.4:
            print(f'SI: {productos[i]} // {n}')
        else:
            print(f'NO: {productos[i]} // {n}')
            precios_correctos.append(precios[i])
    
    if len(precios_correctos)>0:
      lista_precios.append(int(np.mean(precios_correctos)))
      print(f'Encontrado artículo {i+1} de {len(productos_buscable)}\n')
    else:
        print(f'NO encontrado articulo {i} de {len(productos_buscable)}\n')
        no_buscables.append(productos[i])
        del productos[i]
        del precio_compra[i]
        
  else:
    print(f'NO encontrado articulo {i} de {len(productos_buscable)}\n')
    no_buscables.append(productos[i])
    del productos[i]
    del precio_compra[i]
    

# df
ganancia = [lista_precios[i] - precio_compra[:10][i] for i in range(len(lista_precios))]
df_final = pd.DataFrame([productos[:10], ganancia, precio_compra[:10], lista_precios]).T
df_final.columns = ['producto', 'ganancia', 'precio de compra', 'precio de venta']
df_final.sort_values('ganancia', ascending=False, inplace=True)

SI: Ropa Gorra (estampa)  // Body Estampado Bebé Primera Navidad Personalizado Nombre
SI: Ropa Gorra (estampa)  // Gorra Gorro Tactica F1 Estampada Ripstop Antidesgarro Negra
NO: Ropa Gorra (estampa)  // Ajuar 5 Piezas Bebé Recién Nacido + Caja Liso O Estampado
NO: Ropa Gorra (estampa)  // Plancha Sublimadora Portátil 2en1  Plana 23x23 Cm Y Taza 8cm
SI: Ropa Gorra (estampa)  // Gorra Cubre Nuca Proteccion Uv Bebe Pirata
SI: Ropa Gorra (estampa)  // Gorras Estampadas Paramedico
SI: Ropa Gorra (estampa)  // Promo Fiesta - Gorra Truckers Camionera - Estampados Gratis 
NO: Ropa Gorra (estampa)  // Cuello Térmico Multifunción - Buff - Hydrowick® - Alfest®
SI: Ropa Gorra (estampa)  // Gorras Truckers Camioneras Fiestas Oferta Estampado Gratis
SI: Ropa Gorra (estampa)  // Gorra Cubre Nuca Proteccion Uv Bebe Mariposa
SI: Ropa Gorra (estampa)  // Gorrito Gorro Bebe Beba Con Visera Diseño Patitas Estampadas
SI: Ropa Gorra (estampa)  // Gorra Cubre Nuca Proteccion Uv Bebe Dino
SI: Ropa Gorra (est

In [118]:
precios

[3900, 2599, 77990]

In [102]:
r = requests.get(base_url+prod)
soup = BeautifulSoup(r.content, 'html.parser')

In [106]:
nombres = soup.find_all('h2', {'class':'ui-search-item__title shops__item-title'})
nombres = [n.text for n in nombres]

for i, n in enumerate(nombres):
    if string_similarity(productos[i], n) > 0.2:
        print(f'SI: {n}')
    else:
        print(f'NO: {n}')
        del precios[i]

SI: Carnebolic 1,85 Lb Ultimate Nutrition Proteína De Carne 0% Azucar 0% Lactosa
SI: Ultimate Nutrition Carnebolic 1.85 Lb Pura Proteina De Carne
SI: Ultimate Nutrition Carnebolic 1.85 Lb Pura Proteina De Carne
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 4lb | Vanilla
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 4lb | Choc
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Lemon
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 4lb | Choc
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Lemon
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Vanilla
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Orange
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Bl Rasp
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Vanilla
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 4lb | Orange
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Choc
SI: Ultimate Nutrition | Carnebolic Beef Isolate | 2lb | Orange
SI: Ultimate Nutrition | 

In [109]:
# !pip install editdistance
# import editdistance

def string_similarity(str1, str2):
    str1 = str1.strip().lower()
    str2 = str2.strip().lower()
    set1 = set(str1)
    set2 = set(str2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    similarity = len(intersection) / len(union)
    return similarity

In [100]:
a = 'Sublimadora Estampadora 5 En 1 29x38cm Multifunción Plancha'
b = 'Ropa Gorra (estampa)'
string_similarity(a, b)

0.23529411764705882